In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install -q keras

In [2]:
!pip install scikit-learn==0.21.2

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Churn_Modelling.csv')

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
dumie1=pd.get_dummies(df['Geography'],drop_first=True)
dumie2=pd.get_dummies(df['Gender'],drop_first=True)

In [0]:
df=pd.concat([df,dumie1,dumie2],axis=1)

In [8]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain,Male
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [0]:
y=df['Exited']
df=df.drop(['Geography','Gender','Exited'],axis =1)

In [0]:
x=df.iloc[:,3:14]

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [0]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [0]:
X_train=sc.fit_transform(x_train)
X_test=sc.transform(x_test)

In [14]:
import keras

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,LeakyReLU,BatchNormalization,Flatten,Dropout
from keras.activations import sigmoid,relu

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
def creating_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=x_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [0]:
model=KerasClassifier(build_fn=creating_model,verbose=0)

In [0]:
layers=[[20],[20,40],[45,30,15]]
activations=['sigmoid','relu']
param_grid=dict(layers=layers,activation=activations,batch_size=[150,300],epochs=[40])

In [0]:
grid=GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [0]:
grid_result=grid.fit(X_train,y_train)

In [24]:
print(grid_result.best_score_,grid_result.best_params_)
pred_y=grid.predict(X_test)
y_pred=(pred_y>0.5)

0.854714298248291 {'activation': 'relu', 'batch_size': 150, 'epochs': 40, 'layers': [45, 30, 15]}


In [0]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [26]:
cm

array([[2263,  116],
       [ 310,  311]])

In [0]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

In [28]:
score

0.858